In [3]:
import pandas as pd 
import numpy as np 
import re
import requests
from bs4 import BeautifulSoup

Get all movies from the Bechdel dataset, containing Title, IMDb id's, year of release and the Bechdel-test scores. 
Called via the API, it's a little too easy. If we wanted to know more about the submitters and comments, we could use the more elaborate calls. 

In [28]:
bechdel_films = pd.read_json('http://bechdeltest.com/api/v1/getAllMovies') 

Write it into a csv, so we won't screw up with the Bechdel API curators. They kindly ask us to cache and not call the getAllMovies too often. 

In [51]:
bechdel_films.to_csv('data/bechdel_films.csv')

In [4]:
bechdel_films = pd.read_csv('data/bechdel_films.csv' , dtype = str).drop("Unnamed: 0" , 1 ) #read as sting, otherwise our prescious imdb id's are float types

/var/folders/pf/1ppt1rjs6h3dwdsjkpzmyrqc0000gn/T/ipykernel_68040/464398464.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  bechdel_films = pd.read_csv('data/bechdel_films.csv' , dtype = str).drop("Unnamed: 0" , 1 ) #read as sting, otherwise our prescious imdb id's are float types


In [157]:
bechdel_films

,title,id,year,imdbid,rating
0,Passage de Venus,9602,1874,3155794,0
1,La Rosace Magique,9804,1877,14495706,0
2,Sallie Gardner at a Gallop,9603,1878,2221420,0
3,Le singe musicien,9806,1878,12592084,0
4,Athlete Swinging a Pick,9816,1881,7816420,0
...,...,...,...,...,...
9412,Being The Ricardos,10199,2021,4995540,3
9413,Boiling Point,10200,2021,11127680,3
9414,"Power of the Dog, The",10202,2021,10293406,0
9415,Mass,10206,2021,11389748,3


First, let's get the tags/keywords to figure out which film is LGBTQ+ related.  
We scrape every film that is in the Bechdel-test dataset.  
Save the full lists of tags into a dictionary by IMDb id's for later use.  
Later scraping steps are also going to be saved into dics by IMDb id's, so we can join them. 

In [83]:
tag_dict = {}

In [177]:
j = 0
for i in bechdel_films['imdbid']:
    try: 
        res = requests.get("https://www.imdb.com/title/tt" + i + "/keywords?ref_=tt_stry_kw")
        soup = BeautifulSoup(res.text, 'html.parser')
        taglist = soup.find_all('div' , class_="sodatext")
        tag_dict[i] = [ j.text.strip('\n') for j in taglist ]
        j = j + 1 
        print( str(j) + ":" + i)
    except (ConnectionError , TypeError) as err: 
        #if there is a connection or type error, then print error msg it with the ID, so the code doesn't get interrupted.
        print(err + ": IMDb ID:" + i )

IndentationError: unexpected indent (1283662324.py, line 3)

Save our precious dictionary into a file, then load it back

In [113]:
np.save('data/tag_dictionary.npy', tag_dict)

In [5]:
tag_dict = np.load('data/tag_dictionary.npy', allow_pickle = True ).item()

Let's define our LGBTQ keywords.    
By checking if a film is LGBTQ+ related, we iterate through the tags/keywords scraped from IMDb and check how many of the keywords (cleaned from punctiation, special characters) is in the the LGBTQ+ keyword list.  
We save it into a dataframe, with following columns: IMDb ID (for joins), number of keywords scraped, number of LGBTQ+ keyword spotted.  

In [172]:
lgbtq_keywords = [ 'lgbt' , 'lgbtq' ,
                  'gay' , 'gays' , 'gaycharacter' , 'gaykiss' , 'gayinterest' , 
                  'lesbian' , 'lesbians' , 'lesbianinterest' , 
                  'queer' , 
                  'trans' , 'transsexual' , 'transwoman' , 'transman' , 'transgender' , 
                  'bisexual']

In [173]:
df_rows = []
for k,v in tag_dict.items():
    isitgay = sum([ (re.sub( r'\W+', '', vv) in lgbtq_keywords) for vv in v])
    df_rows.append( [ k , isitgay , len(v) ] )  

In [174]:
gay_df = pd.DataFrame(df_rows, columns = ['imdbid','lgbtq_keywords_num','keywords_num'])

In [175]:
df = pd.merge( bechdel_films , gay_df , how = 'inner')

In [176]:
df[ df['lgbtq_keywords_num']  > 0 ]

,title,id,year,imdbid,rating,lgbtq_keywords_num,keywords_num
52,Dickson Experimental Sound Film,9654,1894,0177707,0,2,16
123,"Eclipse: Courtship of the Sun and Moon, The",5223,1907,0215737,0,2,12
137,A Florida Enchantment,4457,1914,0003973,2,4,28
211,Wings,3118,1927,0018578,1,4,73
213,Downhill [When Boys Leave Home],7324,1927,0017825,1,2,74
...,...,...,...,...,...,...,...
9399,West Side Story,10157,2021,3581652,3,1,178
9401,Single All The Way,10161,2021,14315756,3,4,55
9407,Licorice Pizza,10185,2021,11271038,3,1,321
9410,tick...tick..BOOM!,10190,2021,8721424,3,4,100


In [178]:
df.to_csv('data/bechdel_films_ifgay.csv')